In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1668779120784_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.conf import SparkConf
from pyspark.sql.functions import sha2, concat_ws
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
conf = SparkConf()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Fetching the app config path
conf.get("spark.path")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://shibi-lz-07/configuration/app-config.json'

In [5]:
# Converting it into the spark dataframe
appconfig_df=spark.read.option("multiline","true").format("JSON").load(conf.get("spark.path"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Got the individual path respectively
appconfig_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+
|             actives|       final-actives|    final-viewership|          viewership|
+--------------------+--------------------+--------------------+--------------------+
|[[s3://shibi-rz-0...|[[s3://shibi-sz-0...|[[s3://shibi-sz-0...|[[s3://shibi-rz-0...|
+--------------------+--------------------+--------------------+--------------------+

In [16]:
class SparkJob:
    def __init__(self,LandingPath,RawPath):
        self.LandingPath=LandingPath
        self.RawPath=RawPath
    def LandingToRaw(self):
        spark.read.parquet(self.LandingPath).write.mode('overwrite').parquet(self.RawPath)
        rawdata=spark.read.parquet(self.RawPath)
        return rawdata
    def masking(self,data,columns):
        for column in columns:
            masked_data=data.withColumn("masked_"+column, sha2(data[column], 256))
            print('masking is done')
        return masked_data
    def transformation(self,data,precision,*columns):
        for column in columns:
            transformed_data=data.withColumn(column, data[column].cast(DecimalType(precision)))
            print('transformation is done')
        return transformed_data
    def partitionandstaging(self,data,stazing_zone_path):
        data.withColumn("year", year(col("date"))).withColumn("day", dayofmonth(col("date"))).write.partitionBy('year','month','day').mode('Overwrite').parquet(stazing_zone_path)
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# for i in range(5):
#     print(appconfig_df.first()['final-actives'][i])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# appconfig_df.first()['final-viewership'][4][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# appconfig_df.first()['final-actives'][0][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# obj=SparkJob(appconfig_df.first()[i][1][0])
# rawdata=obj.readRawData()
# appconfig_df.first()['final-'+i][4][0][0]
# appconfig_df.first()['final-'+i][4][0]
# appconfig_df.first()['final-'+i][1]
appconfig_df.first()['actives'][0][0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://shibi-rz-07/actives'

In [17]:
datasets = ['actives','viewership']

for i in datasets:
    print(i)
    obj=SparkJob(appconfig_df.first()[i][1][0],appconfig_df.first()[i][0][0])
    try:
        obj.LandingToRaw()
    except:
        print('No path existed for '+i)
        continue
    
    rawdata=obj.LandingToRaw()
#     for column in appconfig_df.first()['final-'+i][1]:
    maskeddata=obj.masking(rawdata,appconfig_df.first()['final-'+i][1])
    transformeddata=obj.transformation(maskeddata,int(appconfig_df.first()['final-'+i][4][1]),appconfig_df.first()['final-'+i][4][0][0],appconfig_df.first()['final-'+i][4][0][1])
    obj.partitionandstaging(transformeddata,appconfig_df.first()['final-'+i][0][0])
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

actives
DataFrame[advertising_id: string, city: string, location_category: string, location_granularities: string, location_source: string, state: string, timestamp: string, user_id: string, user_latitude: double, user_longitude: double, date: timestamp, month: string]
masking is done
masking is done
transformation is done
transformation is done
viewership
DataFrame[advertising_id: string, channel_genre: string, channel_name: string, city: string, device: string, device_type: string, duration: bigint, grid_id: string, language: string, location_category: string, location_granularities: string, location_source: string, record_timestamp: string, show_genre: string, show_name: string, state: string, user_lat: double, user_long: double, date: timestamp, month: string]
masking is done
transformation is done
transformation is done